In [1]:
import time
import re
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import requests
from selenium import webdriver

In [2]:
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=True)
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

In [3]:
title_tags = soup.find_all('div', class_='content_title')
# for some reason soup.find was not returning the most recent so
# get the first element in the list of content titles. 
news = title_tags[0]

news_title = news.get_text()
print(news_title)

# next item is the teaser body
teaser = news.next_sibling
news_p = teaser.get_text()
print(news_p)

NASA Garners 7 Webby Award Nominations
Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.


In [4]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

browser.click_link_by_partial_text('FULL IMAGE')
# wait for the page to load
time.sleep(2)
browser.click_link_by_partial_text('more info')
# wait for the page to load
time.sleep(2)
browser.click_link_by_partial_text('.jpg')

In [143]:
html = browser.html
soup = bs(html, 'html.parser')

featured_img_url = soup.find('img').get('src')
print(featured_img_url)

https://photojournal.jpl.nasa.gov/jpeg/PIA17254.jpg


In [5]:
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)
html = browser.html

soup = bs(html, "html.parser")
tweets = soup.find_all("p", "TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")

mars_weather = tweets[0].get_text()
mars_weather

'That #BlackHole image that’s all over your twitter feed, it’s pretty big. Randall Munroe of xkcd puts it into perspective \n\nhttps://xkcd.com/2135/\xa0pic.twitter.com/3cmwplcIBZ'

In [6]:
url = 'https://space-facts.com/mars/'
table_df = pd.read_html(url)[0]
table_df = table_df.rename(columns={0:'Mars Planet Profile', 1:''})
table_df = table_df.set_index('Mars Planet Profile', drop=True)
table_df

,
Mars Planet Profile,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [145]:
table = table_df.to_html(classes = 'table table-striped')
print(table)

<table border="1" class="dataframe table table-striped">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
    </tr>
    <tr>
      <th>Mars Planet Profile</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.42 x 10^23 kg (10.7% Earth)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.52 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-153 to 20 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [8]:
base_url = 'https://astrogeology.usgs.gov'
list_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(list_url)

list_html = browser.html
soup = bs(list_html, 'html.parser')

In [9]:
hem_img_urls = []

links = soup.find_all('a', class_="itemLink product-item")

def find_hemisphere_name(link_text):
    end_index = link_text.find("Enhanced", 0)
    #take the string up to the index right before "Enhanced"
    return link_text[0:(end_index - 1)]

for link in links:
    #there are links with no text in the html body that are duplicates of the "Enhanced" links, 
    link_text = link.get_text()
    if "Enhanced" in link_text:
        sub_url = f"{base_url}{link.get('href')}"
        
        hem_name = find_hemisphere_name(link_text)
        
        #visit the page for this hemisphere and load its HTML
        browser.visit(sub_url)
        sub_soup = bs(browser.html, 'html.parser')
        
        # find the link to the high def image by finding the link that says "Original" on the page
        original_link = sub_soup.find("a", string="Original")
        original_image_url = original_link.get("href")
        
        # add the results to our url list
        hem_dict = {}
        hem_dict["title"] = hem_name
        hem_dict["img_url"] = original_image_url
        hem_img_urls.append(hem_dict)

print(hem_img_urls)

[{'title': 'Cerberus Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]
